In [66]:
import numpy as np
from numpy.linalg import det, inv
from scipy.special import digamma
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

### Load Data

In [67]:
data = pd.read_csv("data/last_fm_songs_with_features.csv", delimiter=';')
data.drop(['artist_name_lastfm', "tra_id", "Unnamed: 0", 'title_lastfm', 'track_id'], axis=1, inplace=True)

data['genre'] = data['genre'].astype('category').cat.codes

data

,genre,artist_name,title,loudness,tempo,time_signature,key,mode,duration,avg_timbre1,...,var_timbre3,var_timbre4,var_timbre5,var_timbre6,var_timbre7,var_timbre8,var_timbre9,var_timbre10,var_timbre11,var_timbre12
0,3,Abigail Washburn,Sometimes,-6.937,111.510,1,2,1,171.36281,45.124333,...,666.118018,728.287176,465.029834,654.257658,351.989148,254.611756,241.334072,200.432008,157.329807,205.699924
1,3,Abigail Washburn,Red & Blazing,-12.666,117.243,1,9,1,250.67057,36.711987,...,1255.027706,1128.480883,1032.767415,560.339289,730.951534,287.235598,447.818565,214.382203,277.242375,560.245773
2,3,Abigail Washburn,Single Drop Of Honey,-17.884,40.815,1,9,1,163.73506,28.133561,...,4606.643746,9308.677142,2008.470016,2090.427589,2184.097256,1301.974154,514.824958,916.200846,1554.431479,647.123507
3,6,Aborted,A Cold Logistic Slaughter,-6.450,126.372,1,9,1,132.30975,49.039832,...,636.624601,1829.881745,560.364525,748.960503,263.815567,308.850406,251.735934,218.323933,240.353664,117.877781
4,6,Aborted,The Chondrin Enigma,-5.652,108.055,4,3,0,259.73506,48.967769,...,1595.483272,496.593449,316.731837,331.305344,205.525393,301.011338,245.602417,146.633741,133.830178,103.826498
5,6,Aborted,Underneath Rorulent Soil,-6.075,120.352,4,3,0,290.66404,48.318453,...,648.781289,265.432945,493.502503,228.434942,321.490991,177.136525,533.988362,158.772734,116.911361,366.840763
6,6,Aborted,Hecatomb,-6.304,125.213,1,2,1,169.32526,49.638846,...,487.415125,834.854137,472.462584,529.836649,229.980154,232.798783,289.479489,189.001360,208.129047,101.863824
7,6,Aborted,The Gangrenous Epitaph,-4.980,120.046,4,7,1,204.48608,51.738476,...,359.106378,974.179574,288.839429,450.599466,254.623749,233.962651,281.921244,169.962321,133.213200,136.040543
8,6,Aborted,A Methodical Overture,-5.008,110.120,4,6,1,204.19873,51.586782,...,309.652944,734.024311,353.142079,366.783369,214.164046,238.896723,193.253469,152.204371,103.285874,97.456117
9,6,Aborted,The Inertia,-5.865,158.521,3,0,0,214.77832,50.783863,...,419.169448,449.942216,286.209477,285.105570,221.617483,237.547477,233.426137,136.197733,148.085369,130.881752


In [68]:
# Map time 't' to a vector. The epsilon below is the one from the paper
def vectorize(t, epsilon):
#     print(t)
#     print(epsilon.shape)
    v = np.concatenate([t - epsilon, [t,1]])
    v[np.where(v < 0)] = 0
    
    return v

epsilon = np.array([np.power(2,i) for i in np.arange(-3,12,dtype=float)])
vectorize(5, epsilon)

array([4.875, 4.75 , 4.5  , 4.   , 3.   , 1.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 5.   , 1.   ])

### Create Song x Feature Matrix

In [69]:
data_x = data.drop(['artist_name', 'title'], axis=1).values
num_songs, p = data_x.shape

### Driver

In [70]:
current = np.datetime64(datetime.datetime.now())
last_listened = np.array([current]*num_songs)

In [85]:
def calculate_quantile(x,t,x_mean_coeff, x_var_coeff, t_mean_coeff, t_var_coeff, alpha, num_samples):
    # sample from theta.T.dot(x) and beta.T.dot(x) and create histogram
    samples_1 = np.random.normal(x.T.dot(x_mean_coeff).item(), x.T.dot(x_var_coeff).dot(x).item(), size = num_samples)
    samples_2 = np.random.normal(t.T.dot(t_mean_coeff).item(), t.T.dot(t_var_coeff).dot(t).item(), size = num_samples)
    
    final_samples = samples_1*samples_2
    
    # construct histogram
    pdf, intervals = np.histogram(final_samples, 100)
    cdf = np.cumsum(pdf/sum(pdf))
    
    # return quantile 
    for index, value in enumerate(cdf):
        if value >= alpha:
            return intervals[index]

In [74]:
class Bayes_UCB_V:
    def __init__(self, x, t, r):
        self.THRESHOLD = 10e-3
        self.x = x # features of songs - dim : num_of_features x num_of_songs
        self.t = t # last listened times of songs - dim : (num_of_intervals + 2) x num_of_songs
        self.r = r # ratings - dim : 1 x num_of_songs
        self.p,self.N = x.shape
        self.K = t.shape[0]
        # TODO: How to initialize these variables?
        self.lambda_theta_N = np.identity(self.p)
        self.eta_theta_N = np.zeros((self.p,1))
        self.eta_beta_N = np.zeros((self.K,1))
        self.lambda_beta_N = np.identity(self.K)
        self.D_0 = np.identity(self.p)
        self.E_0 = np.identity(self.K)
        self.mu_theta_0 = np.zeros((self.p,1))
        self.mu_beta_0 = np.zeros((self.K,1))
        self.a_0 = 2
        self.b_0 = 2 * 10e-8
        self.a_N = self.a_0
        self.b_N = self.b_0
        self.prev_L = 0.0

        
    def optimize_parameters(self):
        while self.is_converged(self.lower_bound()) == False:
            self.update_theta()
            self.update_beta()
            self.update_tau()
        return self.lambda_theta_N, self.eta_theta_N, self.lambda_beta_N, self.eta_beta_N
    
    def lower_bound(self):       
        return (self.a_0 * np.log(self.b_0) 
            + (self.a_0 - 1) * (digamma(self.a_N) - np.log(self.b_N))
            - (self.b_0 * self.a_N / self.b_N)
            - (np.log(2 * np.pi)/2)
            - ( np.log(det(self.D_0))/2 )
            + (self.p / 2) * (digamma(self.a_N) - np.log(self.b_N))
            - (self.a_N / (2 * self.b_N)) * (np.trace(self.D_0.dot(inv(self.lambda_theta_N))) 
                                               + (self.mu_theta_0 - self.expected_theta()).T.dot(inv(self.D_0)).dot(self.mu_theta_0 - self.expected_theta())
                                             )
            - (self.K * np.log(2 * np.pi) / 2)
            - (np.log(det(self.E_0)) / 2)
            + (self.K / 2) * (digamma(self.a_N) - np.log(self.b_N))
            - (self.a_N / (2 * self.b_N)) * (np.trace(self.E_0.dot(inv(self.lambda_beta_N))) 
                                               + (self.mu_theta_0 - self.expected_theta()).T.dot(inv(self.D_0)).dot(self.mu_theta_0 - self.expected_theta())
                                             )
            - (np.log(2 * np.pi)/ 2)
            + (1 / 2) * (digamma(self.a_N) - np.log(self.b_N))
            - (self.a_N / (2 * self.b_N)) * self.sum1() # Check
            + (self.a_N / self.b_N) * self.sum2() # Check
            + (self.K / 2) * (1 + np.log(2 * np.pi))
            + (1 / 2) * (np.log(det(inv(self.lambda_beta_N))))
            + (self.p / 2) * (1 + np.log(2 * np.pi))
            + (1 / 2) * (np.log(det(inv(self.lambda_theta_N))))
            - (self.a_N - 1) * digamma(self.a_N)
            - np.log(self.b_N) + self.a_N        
            )        
                       
    def sum1(self):
        result = self.r.dot(self.r.T)

        for i in range(self.N):
            x_i = self.x[:,i].reshape((self.p,1))
            t_i = self.t[:,i].reshape((self.K,1))
            
            result += x_i.T.dot(self.expected_outer_theta()).dot(x_i).dot(t_i.T).dot(self.expected_outer_beta()).dot(t_i)
            
        return result
    
    def sum2(self):
        result = 0
        
        for i in range(self.N):
            x_i = self.x[:,i].reshape((self.p,1))
            t_i = self.t[:,i].reshape((self.K,1))
            
            r_i = self.r[0][i]
            
            result += r_i*(x_i.T.dot(self.expected_theta()).dot(t_i.T).dot(self.expected_beta())) + self.b_0
            
        return result
                       
    def is_converged(self, L):
        if abs(L - self.prev_L) < self.THRESHOLD:
            return True
        else:
            self.prev_L = L
            return False
    
    def sum_product_along_axis_1(self, First, Second, Beta):
        p,N = First.shape
        K = Second.shape[0]

        result = np.zeros((p,p))
        
        for i in range(N):
            f_i = First[:,i].reshape((p,1))
            s_i = Second[:,i].reshape((K,1))
            
            result += f_i.dot(s_i.T).dot(Beta).dot(s_i).dot(f_i.T)
            
        return result
    
    def sum_product_along_axis_2(self, v_1, First, Second, v_2):
        p,N = First.shape
        K = Second.shape[0]

        result = np.zeros((p,1))
        
        for i in range(N):
            f_i = First[:,i].reshape((p,1))
            s_i = Second[:,i].reshape((K,1))
            r_i = v_1[0][i]
            
            result += ( r_i * (f_i.dot(s_i.T).dot(v_2)) )
            
        return result
        
    def update_theta(self):
        exp_tau = self.expected_tau()
        
        self.lambda_theta_N = exp_tau * (inv(self.D_0) + self.sum_product_along_axis_1(self.x, self.t, self.expected_outer_beta()))

        self.eta_theta_N = exp_tau * (inv(self.D_0).dot(self.mu_theta_0)  + self.sum_product_along_axis_2(self.r, self.x, self.t, self.expected_beta()))

    def update_beta(self):               
        exp_tau = self.expected_tau()
        E_0_inv = inv(self.E_0)

        self.lambda_beta_N = exp_tau * (E_0_inv + self.sum_product_along_axis_1(self.t, self.x, self.expected_outer_theta()))              
        
        self.eta_beta_N = exp_tau * ( E_0_inv.dot(self.mu_beta_0) + self.sum_product_along_axis_2(self.r, self.t, self.x, self.expected_theta()))

    def update_tau(self):
        self.a_N = (self.p + self.K + self.N) / 2 + self.a_0
        b_N_update = 0.5*(np.trace(inv(self.D_0).dot(self.expected_outer_theta())) + (self.mu_theta_0.T - 2*self.expected_theta().T).dot(inv(self.D_0)).dot(self.mu_theta_0))

        b_N_update += 0.5*(np.trace(inv(self.E_0).dot(self.expected_outer_beta())) + (self.mu_beta_0.T - 2*self.expected_beta().T).dot(inv(self.E_0)).dot(self.mu_beta_0))
        
        b_N_update += (0.5*self.sum1() - self.sum2() + self.b_0)
        
        self.b_N = b_N_update

    def expected_beta(self):
        return inv(self.lambda_beta_N).dot(self.eta_beta_N)
    
    def expected_theta(self):
        return inv(self.lambda_theta_N).dot(self.eta_theta_N)
    
    def expected_tau(self):
        return self.a_N / self.b_N
    
    def expected_outer_beta(self):
        exp_beta = self.expected_beta()
        return inv(self.lambda_beta_N) + exp_beta.dot(exp_beta.T)
    
    def expected_outer_theta(self):
        exp_theta = self.expected_theta()
        return inv(self.lambda_theta_N) + exp_theta.dot(exp_theta.T)


In [ ]:
def driver(features, last_listened, ratings, epsilon):
    time_vectors = np.zeros((len(last_listened),len(epsilon) + 2))
    for i, x in enumerate(last_listened):
        time_vectors[i,:] = vectorize((np.datetime64(datetime.datetime.now()) - x).astype('timedelta64[m]').astype('int'), epsilon)
    
    time_vectors = time_vectors.T
    
    ratings = ratings.reshape((1,len(ratings)))
    
    bayesUCB_V = Bayes_UCB_V(features, time_vectors, ratings)
    
    lambda_theta_N, eta_theta_N, lambda_beta_N, eta_beta_N = bayesUCB_V.optimize_parameters()
    
    # section 4.2.3 : page 11
    x_mean_coeff = inv(lambda_theta_N).dot(eta_theta_N)
    x_var_coeff = inv(lambda_theta_N)
    
    t_mean_coeff = inv(lambda_beta_N).dot(eta_beta_N)
    t_var_coeff = inv(lambda_beta_N)
    
    # get quantiles for each song and choose argmax
    N = features.shape[1]
    quantiles = []
    for i in range(N):
        x_i = features[:,i].reshape((features.shape[0],1))
        t_i = time_vectors[:,i].reshape((time_vectors.shape[0],1))
        
        quantiles.append(calculate_quantile(x_i, t_i, x_mean_coeff, x_var_coeff, t_mean_coeff, t_var_coeff, 1 - 1/(N + 1), 10000))
        
    return np.argmax(quantiles)

In [91]:
index = driver(data_x[:100,:].T, [current]*100, np.ones(100), epsilon)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in log


In [92]:
index

2